# Libraries

In [1]:
import pyodbc
import pandas as pd
import jupyternotify
from datetime import datetime
import pandas.io.formats.excel
pandas.io.formats.excel.ExcelFormatter.header_style = None
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

<IPython.core.display.Javascript object>

# Parámetros

In [2]:
anio = 2021
semana = 29
file = '\\INFOR GENOMMA LAB COLOMBIA JULIO 20 2021.xlsx'
path = r"C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Copservir\Copservir Faltante\2021\S {0}"
path = path.format(semana)

# Importación

In [3]:
%%time
# Carga de Inventario por PDV
sheet_names = ['INV POR PDV', 'inv x pdv', 'INV X PDV']
for sheet in sheet_names:
    try:
        df_ip = pd.read_excel(path + file, sheet_name=sheet)
        break
    except:
        continue
    
# Carga de Venta Diaria
sheet_names = ['vta x dia', 'VTA X DIA', 'VTA POR DIA', 'VTA POR DIA ', 'Sheet1']
for sheet in sheet_names:
    try:
        df_vd = pd.read_excel(path + file, sheet_name=sheet)
        break
    except:
        continue

        # Carga de Inventario de CEDIS
df_ic = pd.read_excel(path + file, sheet_name='INV CONSOLIDADO', header=2)

Wall time: 2min 34s


## DWH

In [4]:
server = 'SFEDWH01'
database1 = 'Gnm_DWH'

conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=' + server + ';'
                     'Database=' + database1 + ';'
                     'Trusted_Connection=yes;')

query_dias = '''SELECT TmpFecha
                FROM Gnm_DWH.dbo.Dim_Tiempo
                WHERE TmpAnioSemanaGenomma = {0} AND TmpSemanaAnioGenomma = {1}'''

In [5]:
df_dias = pd.read_sql(query_dias.format(anio, semana), conn1)

# Transformación de Inventarios

Primero transformamos el *Inventario por PDV (ip)*:

In [6]:
data_ip = df_ip[['Mostrador', 'Código producto', 'Producto', 'Cantidad Saldo']].copy()
data_ip['codHoml'] = df_ip['Mostrador'].apply(lambda x: x.split('-')[0])
data_ip['so'] = 0

Luego, transformamos el *inventario de CEDIS (ic)*:

In [7]:
data_ic = df_ic[['Código producto', 'Producto', 'Cantidad Saldo.1', 'Cantidad Saldo.2', 'Cantidad Saldo.3', 'Cantidad Saldo.4']].copy()
data_ic.rename({'Cantidad Saldo.1':'BAR', 'Cantidad Saldo.2':'BOG', 'Cantidad Saldo.3':'BUC', 'Cantidad Saldo.4':'CAL'}, axis=1, inplace=True)

# Despivoteamos las columnas y lo guardamos en data_ic_t
data_ic_t = data_ic.melt(id_vars=['Código producto', 'Producto'], value_vars=['BAR', 'BOG', 'BUC', 'CAL'])

# Transformación de los datos despivoteados
data_ic_t.rename({'value':'Cantidad Saldo', 'variable':'codHoml'}, axis=1, inplace=True)
data_ic_t['Mostrador'] = data_ic_t['codHoml'].map({'BAR':'BARRANQUILLA', 'BOG':'BOGOTA', 'BUC':'BUCARAMANGA', 'CAL':'CALI'})
data_ic_t['so'] = 0

Y concatenamos ambos inventarios en unos solo:

In [8]:
data_inv = pd.concat([data_ip, data_ic_t])

Y limpiamos algunas columnas

In [9]:
data_inv.loc[data_inv['Cantidad Saldo'] == '-', 'Cantidad Saldo'] = 0

data_inv['Cantidad Saldo'] = data_inv['Cantidad Saldo'].astype(float)

# Transformación de Ventas

Función para obtener el nombre de las columnas según las fechas de la semana en cuestión:

In [10]:
def ftarget(dias):
    
    meses = {"01":'Ene', "02":'Feb', "03":'Mar',
             "04":'Abr', "05":'May', "06":'Jun',
             "07":'Jul', "08":'Ago', "09":'Sep',
             "10":'Oct', "11":'Nov', "12":'Dic'}
    
    fechas_list = []
    range_fechas = dias['TmpFecha'].tolist()
    for f in range_fechas:
        fechas_list.append(str(f.day).zfill(2) + '_' + meses[str(f.month).zfill(2)] + '_' + str(f.year)[-2:])
    return fechas_list

In [11]:
# Obtenemos los nombres de las columnas a tomar
cols_datos = ftarget(df_dias)
#data_vd = df_vd[['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Día'] + cols_datos].copy()
data_vd = df_vd[['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Día'] + [c for c in cols_datos if c in df_vd.columns]].copy()

# Limpiamos
data_vd.replace('-', 0, inplace=True)

# Sumamos los diarios para tener semanal
#data_vd['so'] = data_vd[cols_datos].sum(axis=1)
data_vd['so'] = data_vd[[c for c in cols_datos if c in data_vd.columns]].sum(axis=1)

# Renombramos columnas 
data_vd.rename(dict(zip(data_vd.columns[:-1], data_vd.loc[0].tolist()[:-1])), axis=1, inplace=True)

# Limpiamos
data_vd.drop(['Cantidad'], axis=1, inplace=True)
data_vd.drop(0, axis=0, inplace=True)
data_vd.reset_index(inplace=True, drop=True)
data_vd['Cantidad Saldo'] = 0

# Merge de inventario y stock

Función para homologar los códigos de sucursales

In [12]:
def homolog_codigos(codigo):
    
    dict_locales = {'400000000':'4E8', '4000000':'4E6', '400000':'4E5',
                    '40000':'4E4', '4000':'4E3', '400':'4E2', '40':'4E1',
                    '3000000000':'3E9', '300000000':'3E8', '30000000':'3E7',
                    '3000000':'3E6', '300000':'3E5', '30000':'3E4', 
                    '3000':'3E3', '30':'3E1', '200000000':'2E8', '20000000':'2E7',
                    '200000':'2E5', '20000':'2E4', '200':'2E2',  '20':'2E1',
                    '1000000000':'1E9', '100000000':'1E8', '10000000':'1E7',
                    '100000':'1E5', '10000':'1E4', '1000':'1E3', '100':'1E2', '10':'1E1'}
    
    try:
        return dict_locales[codigo]
    except:
        return codigo

Preparación de los datos de inventario y ventas para el merge

In [13]:
# Limpieza Inventario
data_inv['codHoml'] = data_inv['codHoml'].astype(str)
data_inv['codHoml'] = data_inv['codHoml'].map(homolog_codigos)
data_inv['Código producto'] = data_inv['Código producto'].astype(str)
data_inv['ID'] = data_inv['codHoml'] + data_inv['Código producto']

In [14]:
data_vd = data_vd.pivot_table(index=['CodigoComercial', 'Nombre Mostrador', 'Código producto', 'Nombre Producto', 'so'],
                              values=['so', 'Cantidad Saldo'], aggfunc='sum').reset_index()

In [15]:
# Limpieza Ventas
data_vd['CodigoComercial'] = data_vd['CodigoComercial'].astype(str)
data_vd['CodigoComercial'] = data_vd['CodigoComercial'].map(homolog_codigos)
data_vd['Código producto'] = data_vd['Código producto'].astype(str)
data_vd['ID'] = data_vd['CodigoComercial'] + data_vd['Código producto']

In [16]:
data = pd.merge(data_inv, data_vd, on='ID', how='outer')

Ya hecho el merge, se filtran las columnas de interés

In [17]:
def fillna_another_column(df, col1, col2, new_name):
    df[new_name] = df[col1].fillna(df[col2])
    df.drop([col1, col2], axis=1, inplace=True)
    return df

In [18]:
columns_to_fill = [('Código producto_x', 'Código producto_y', 'Código producto'),
                   ('Cantidad Saldo_x', 'Cantidad Saldo_y', 'Cantidad Saldo'),
                   ('so_y', 'so_x', 'so'), 
                   ('Mostrador', 'Nombre Mostrador', 'Mostrad'), 
                   ('codHoml', 'CodigoComercial', 'codHomol'),
                   ('Producto', 'Nombre Producto', 'Product')]

In [19]:
for c1, c2, nn in columns_to_fill:
    data = fillna_another_column(data, c1, c2, nn)
data.drop(['ID'], axis=1, inplace=True)
data.rename({'Mostrad':'Mostrador', 'codHomol':'codHoml', 'Product':'Producto'}, axis=1, inplace=True)

In [20]:
data.isnull().sum()

Código producto    0
Cantidad Saldo     0
so                 0
Mostrador          0
codHoml            0
Producto           0
dtype: int64

In [21]:
data.shape

(90895, 6)

# Exportación de datos

In [22]:
%%notify -m "Exportación de Copservir "
filename = '{0}_{1}-20211122'.format(anio, str(semana).zfill(2))
data[['codHoml', 'Mostrador', 'Código producto', 'Producto', 'Cantidad Saldo', 'so']].to_excel(path + '\\{}.xlsx'.format(filename), index=False, sheet_name=filename)

<IPython.core.display.Javascript object>

In [23]:
path

'C:\\Users\\jshernandezm\\genommalabinternacional\\Cristian Javier Sanchez Yepez - NoB2B\\Copservir\\Copservir Faltante\\2021\\S 29'